In [1]:
from nltk.tag import CRFTagger
import pandas as pd
from nltk import word_tokenize


In [46]:
df = pd.read_csv('hasil_tes.csv')
df.head()

,No Document,Description,tes
0,1,Sebuah truk tak kuat menanjak di Jalan Daendel...,"['ada luka', 'ada luka', 'ada luka']"
1,3,Setidaknya 600 ekor Tasmanian devil tewas di j...,['ada tewas']
2,7,Empat kendaraan terlibat kecelakaan beruntun d...,"['ada tewas', 'ada tewas', 'ada tewas', 'ada t..."
3,8,Sopir angkot berinisial KM yang terlibat tabra...,"['ada tewas', 'ada tewas']"
4,9,antara truk dan bus pariwisata di Tol Dupak ar...,"['ada meninggal', 'ada luka']"


In [3]:
ct =CRFTagger()
ct.set_model_file('all_indo_man_tag_corpus_model.crf.tagger')
def preprocessing(berita):
    s = berita
    # s = s.lower()
    tokens = word_tokenize(s)
    return tokens

In [4]:
def searchCD(arr):
    kumpul=[]
    for x in range(0, len(arr[0])-1):
        if(arr[0][x][1]=='CD'):
            data=[arr[0][x-1],arr[0][x],arr[0][x+1],x]
            kumpul.append(data)
    
    return kumpul

In [5]:
def cek_CD_depan(arr, indeks):
    for x in range(0, indeks):
        if(arr[0][indeks-x][1] != 'Z' and arr[0][indeks-x][1] != 'CC'):
            if(arr[0][indeks-x][1] == 'CD'):
                return(indeks-x,arr[0][indeks-x][0])
        else:
            return 0
    return 0

In [6]:
def getSentence(indeksIndikator, indeksCD,arr):
    teks=[]

    if(indeksIndikator>indeksCD):
        for x in range(indeksCD,indeksIndikator+1):
            # print(x)
            if(x==indeksIndikator):
                teks.append('indikator')  
            else:
                teks.append(arr[0][x])  

    elif(indeksIndikator<indeksCD):
        for x in range(indeksIndikator,indeksCD+1):
            # print(x)
            if(x==indeksIndikator):
                teks.append('indikator')  
            else:
                teks.append(arr[0][x]) 
            # teks.append(arr[0][x])

    return teks
    
def getSentenceNotCD(indeks1, indeks2,indeksIndikator,arr):
    teks=[]

    if(indeks1>indeks2):
        for x in range(indeks2,indeks1+1):
            if(x==indeksIndikator):
                teks.append('indikator')  
            else:
                teks.append(arr[0][x])  

    elif(indeks1<indeks2):
        for x in range(indeks1,indeks2+1):
            if(x==indeksIndikator):
                teks.append('indikator')  
            else:
                teks.append(arr[0][x]) 
            # teks.append(arr[0][x])

    return teks


In [7]:
def cek_CD_belakang(arr, indeks):
    for x in range(indeks,len(arr[0])):
        # print(x,arr[0][x])
        if([1] != 'Z' and arr[0][x][1] != 'CC'):
            if(arr[0][x][1] == 'CD'):
                return(x,arr[0][x][0])
        else:
            return 0
    return 0

In [8]:
def getTitikLoc(indeksIndikator, indeksRange, arr):
    # print(indeksRange)
    # print("ini", indeksIndikator, indeksRange)

    if(indeksIndikator>indeksRange):
        for x in range(indeksRange,indeksIndikator):
            # print(indeksIndikator-x,arr[0][indeksIndikator-x][0])
            if(indeksIndikator-x!=1):
                if(arr[0][indeksIndikator-x][1] == 'Z' and arr[0][indeksIndikator-x][0] == '.'):
                    return indeksIndikator-x
            else:
                return 0
    elif(indeksIndikator<indeksRange):
        for x in range(indeksIndikator,indeksRange):
            # print(x,indeksRange)
            if(x+1!=indeksRange):
                # print('masuk1',arr[0][x][0],x,indeksRange)
                if(arr[0][x][1] == 'Z' and arr[0][x][0] == '.'):
                    # print('masuk2')
                    return x
            else:
                # print('masuk3',indeksRange)
                return indeksRange

    return 0



In [21]:
def getTitik(indeks,arr):
    hasil=[]
    # print(indeks)
    # print(arr)
    titik1=getTitikLoc(indeks,0,arr)
    # print(titik1)
    titik2=getTitikLoc(indeks,len(arr[0]),arr)-1
    # print(titik2)
    hasil.append(titik1)
    hasil.append(titik2)
    
    return hasil

In [52]:
def struktur1(arr,indeks):
    
    if(indeks+1<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='CD' or arr[0][indeks+1][0]=='Seorang' or arr[0][indeks+1][0]=='Seseorang' or arr[0][indeks+1][0]=='Empat' or arr[0][indeks+1][0]=='Lima' or arr[0][indeks+1][0]=='Enam' or arr[0][indeks+1][0]=='Tujuh' or arr[0][indeks+1][0]=='Delapan' or arr[0][indeks+1][0]=='Sembilan' or arr[0][indeks+1][0]=='Sepuluh')):
        # print('masuk1')
        return arr[0][indeks+1][0]
    elif(indeks-1>=0 
        and (arr[0][indeks-1][1]=='CD' or arr[0][indeks-1][0]=='Seorang' or arr[0][indeks-1][0]=='Seseorang' or arr[0][indeks-1][0]=='Empat' or arr[0][indeks-1][0]=='Lima' or arr[0][indeks-1][0]=='Enam' or arr[0][indeks-1][0]=='Tujuh' or arr[0][indeks-1][0]=='Delapan' or arr[0][indeks-1][0]=='Sembilan' or arr[0][indeks-1][0]=='Sepuluh')):
        # print('masuk12')
        return arr[0][indeks-1][0]
    elif(indeks+2<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB') 
        and (arr[0][indeks+2][1]=='CD' or arr[0][indeks+2][0]=='Seorang' or arr[0][indeks+2][0]=='Seseorang' or arr[0][indeks+2][0]=='Empat' or arr[0][indeks+2][0]=='Lima' or arr[0][indeks+2][0]=='Enam' or arr[0][indeks+2][0]=='Tujuh' or arr[0][indeks+2][0]=='Delapan' or arr[0][indeks+2][0]=='Sembilan' or arr[0][indeks+2][0]=='Sepuluh')):
        return arr[0][indeks+2][0]
    elif(indeks-2>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB') 
        and (arr[0][indeks-2][1]=='CD' or arr[0][indeks-2][0]=='Seorang' or arr[0][indeks-2][0]=='Seseorang' or arr[0][indeks-2][0]=='Empat' or arr[0][indeks-2][0]=='Lima' or arr[0][indeks-2][0]=='Enam' or arr[0][indeks-2][0]=='Tujuh' or arr[0][indeks-2][0]=='Delapan' or arr[0][indeks-2][0]=='Sembilan' or arr[0][indeks-2][0]=='Sepuluh')):
        return arr[0][indeks-2][0]
    elif(indeks+3<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='CD' or arr[0][indeks+3][0]=='Seorang' or arr[0][indeks+3][0]=='Seseorang' or arr[0][indeks+3][0]=='Empat' or arr[0][indeks+3][0]=='Lima' or arr[0][indeks+3][0]=='Enam' or arr[0][indeks+3][0]=='Tujuh' or arr[0][indeks+3][0]=='Delapan' or arr[0][indeks+3][0]=='Sembilan' or arr[0][indeks+3][0]=='Sepuluh')):
        # print('masuk14')
        return arr[0][indeks+3][0]
    elif(indeks-3>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='CD' or arr[0][indeks-3][0]=='Seorang' or arr[0][indeks-3][0]=='Seseorang' or arr[0][indeks-3][0]=='Empat' or arr[0][indeks-3][0]=='Lima' or arr[0][indeks-3][0]=='Enam' or arr[0][indeks-3][0]=='Tujuh' or arr[0][indeks-3][0]=='Delapan' or arr[0][indeks-3][0]=='Sembilan' or arr[0][indeks-3][0]=='Sepuluh')):
        # print('masuk15')
        return arr[0][indeks-3][0]
    elif(indeks+5<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='CD' or arr[0][indeks+4][0]=='Seorang' or arr[0][indeks+4][0]=='Seseorang' or arr[0][indeks+4][0]=='Empat' or arr[0][indeks+4][0]=='Lima' or arr[0][indeks+4][0]=='Enam' or arr[0][indeks+4][0]=='Tujuh' or arr[0][indeks+4][0]=='Delapan' or arr[0][indeks+4][0]=='Sembilan' or arr[0][indeks+4][0]=='Sepuluh')):
        return arr[0][indeks+4][0]
    elif(indeks-4>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='CD' or arr[0][indeks-4][0]=='Seorang' or arr[0][indeks-4][0]=='Seseorang' or arr[0][indeks-4][0]=='Empat' or arr[0][indeks-4][0]=='Lima' or arr[0][indeks-4][0]=='Enam' or arr[0][indeks-4][0]=='Tujuh' or arr[0][indeks-4][0]=='Delapan' or arr[0][indeks-4][0]=='Sembilan' or arr[0][indeks-4][0]=='Sepuluh')):
        return arr[0][indeks-4][0]
    elif(indeks+5<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='CD' or arr[0][indeks+5][0]=='Seorang' or arr[0][indeks+5][0]=='Seseorang' or arr[0][indeks+5][0]=='Empat' or arr[0][indeks+5][0]=='Lima' or arr[0][indeks+5][0]=='Enam' or arr[0][indeks+5][0]=='Tujuh' or arr[0][indeks+5][0]=='Delapan' or arr[0][indeks+5][0]=='Sembilan' or arr[0][indeks+5][0]=='Sepuluh')):
        return arr[0][indeks+5][0]
    elif(indeks-5>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='CD' or arr[0][indeks-5][0]=='Seorang' or arr[0][indeks-5][0]=='Seseorang' or arr[0][indeks-5][0]=='Empat' or arr[0][indeks-5][0]=='Lima' or arr[0][indeks-5][0]=='Enam' or arr[0][indeks-5][0]=='Tujuh' or arr[0][indeks-5][0]=='Delapan' or arr[0][indeks-5][0]=='Sembilan' or arr[0][indeks-5][0]=='Sepuluh')):
        return arr[0][indeks-5][0]
    elif(indeks+6<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='CD' or arr[0][indeks+6][0]=='Seorang' or arr[0][indeks+6][0]=='Seseorang' or arr[0][indeks+6][0]=='Empat' or arr[0][indeks+6][0]=='Lima' or arr[0][indeks+6][0]=='Enam' or arr[0][indeks+6][0]=='Tujuh' or arr[0][indeks+6][0]=='Delapan' or arr[0][indeks+6][0]=='Sembilan' or arr[0][indeks+6][0]=='Sepuluh')):
        return arr[0][indeks+6][0]
    elif(indeks-6>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='CD' or arr[0][indeks-6][0]=='Seorang' or arr[0][indeks-6][0]=='Seseorang' or arr[0][indeks-6][0]=='Empat' or arr[0][indeks-6][0]=='Lima' or arr[0][indeks-6][0]=='Enam' or arr[0][indeks-6][0]=='Tujuh' or arr[0][indeks-6][0]=='Delapan' or arr[0][indeks-6][0]=='Sembilan' or arr[0][indeks-6][0]=='Sepuluh')):
        return arr[0][indeks-6][0]
    elif(indeks+7<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='CD' or arr[0][indeks+7][0]=='Seorang' or arr[0][indeks+7][0]=='Seseorang' or arr[0][indeks+7][0]=='Empat' or arr[0][indeks+7][0]=='Lima' or arr[0][indeks+7][0]=='Enam' or arr[0][indeks+7][0]=='Tujuh' or arr[0][indeks+7][0]=='Delapan' or arr[0][indeks+7][0]=='Sembilan' or arr[0][indeks+7][0]=='Sepuluh')):
        return arr[0][indeks+7][0]
    elif(indeks-7>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='CD' or arr[0][indeks-7][0]=='Seorang' or arr[0][indeks-7][0]=='Seseorang' or arr[0][indeks-7][0]=='Empat' or arr[0][indeks-7][0]=='Lima' or arr[0][indeks-7][0]=='Enam' or arr[0][indeks-7][0]=='Tujuh' or arr[0][indeks-7][0]=='Delapan' or arr[0][indeks-7][0]=='Sembilan' or arr[0][indeks-7][0]=='Sepuluh')):
        return arr[0][indeks-7][0]
    elif(indeks+8<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='CD' or arr[0][indeks+8][0]=='Seorang' or arr[0][indeks+8][0]=='Seseorang' or arr[0][indeks+8][0]=='Empat' or arr[0][indeks+8][0]=='Lima' or arr[0][indeks+8][0]=='Enam' or arr[0][indeks+8][0]=='Tujuh' or arr[0][indeks+8][0]=='Delapan' or arr[0][indeks+8][0]=='Sembilan' or arr[0][indeks+8][0]=='Sepuluh')):
        return arr[0][indeks+8][0]
    elif(indeks-8>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='CD' or arr[0][indeks-8][0]=='Seorang' or arr[0][indeks-8][0]=='Seseorang' or arr[0][indeks-8][0]=='Empat' or arr[0][indeks-8][0]=='Lima' or arr[0][indeks-8][0]=='Enam' or arr[0][indeks-8][0]=='Tujuh' or arr[0][indeks-8][0]=='Delapan' or arr[0][indeks-8][0]=='Sembilan' or arr[0][indeks-8][0]=='Sepuluh')):
        return arr[0][indeks-8][0]
    elif(indeks+9<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='CD' or arr[0][indeks+9][0]=='Seorang' or arr[0][indeks+9][0]=='Seseorang' or arr[0][indeks+9][0]=='Empat' or arr[0][indeks+9][0]=='Lima' or arr[0][indeks+9][0]=='Enam' or arr[0][indeks+9][0]=='Tujuh' or arr[0][indeks+9][0]=='Delapan' or arr[0][indeks+9][0]=='Sembilan' or arr[0][indeks+9][0]=='Sepuluh')):
        return arr[0][indeks+9][0]
    elif(indeks-9>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='CD' or arr[0][indeks-9][0]=='Seorang' or arr[0][indeks-9][0]=='Seseorang' or arr[0][indeks-9][0]=='Empat' or arr[0][indeks-9][0]=='Lima' or arr[0][indeks-9][0]=='Enam' or arr[0][indeks-9][0]=='Tujuh' or arr[0][indeks-9][0]=='Delapan' or arr[0][indeks-9][0]=='Sembilan' or arr[0][indeks-9][0]=='Sepuluh')):
        return arr[0][indeks-9][0]
    elif(indeks+10<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='VB' or arr[0][indeks+9][1]=='IN' or arr[0][indeks+9][1]=='NN' or arr[0][indeks+9][1]=='NNP' or arr[0][indeks+9][1]=='SC' or arr[0][indeks+9][1]=='JJ' or arr[0][indeks+9][1]=='PR'  or arr[0][indeks+9][1]=='NND'  or arr[0][indeks+9][1]=='RB') 
        and (arr[0][indeks+10][1]=='CD' or arr[0][indeks+10][0]=='Seorang' or arr[0][indeks+10][0]=='Seseorang' or arr[0][indeks+10][0]=='Empat' or arr[0][indeks+10][0]=='Lima' or arr[0][indeks+10][0]=='Enam' or arr[0][indeks+10][0]=='Tujuh' or arr[0][indeks+10][0]=='Delapan' or arr[0][indeks+10][0]=='Sembilan' or arr[0][indeks+10][0]=='Sepuluh')):
        return arr[0][indeks+10][0]
    elif(indeks-10>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='VB' or arr[0][indeks-9][1]=='IN' or arr[0][indeks-9][1]=='NN' or arr[0][indeks-9][1]=='NNP' or arr[0][indeks-9][1]=='SC' or arr[0][indeks-9][1]=='JJ' or arr[0][indeks-9][1]=='PR'  or arr[0][indeks-9][1]=='NND'  or arr[0][indeks-9][1]=='RB') 
        and (arr[0][indeks-10][1]=='CD' or arr[0][indeks-10][0]=='Seorang' or arr[0][indeks-10][0]=='Seseorang' or arr[0][indeks-10][0]=='Empat' or arr[0][indeks-10][0]=='Lima' or arr[0][indeks-10][0]=='Enam' or arr[0][indeks-10][0]=='Tujuh' or arr[0][indeks-10][0]=='Delapan' or arr[0][indeks-10][0]=='Sembilan' or arr[0][indeks-10][0]=='Sepuluh')):
        return arr[0][indeks-10][0]
    elif(indeks+11<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='VB' or arr[0][indeks+9][1]=='IN' or arr[0][indeks+9][1]=='NN' or arr[0][indeks+9][1]=='NNP' or arr[0][indeks+9][1]=='SC' or arr[0][indeks+9][1]=='JJ' or arr[0][indeks+9][1]=='PR'  or arr[0][indeks+9][1]=='NND'  or arr[0][indeks+9][1]=='RB') 
        and (arr[0][indeks+10][1]=='VB' or arr[0][indeks+10][1]=='IN' or arr[0][indeks+10][1]=='NN' or arr[0][indeks+10][1]=='NNP' or arr[0][indeks+10][1]=='SC' or arr[0][indeks+10][1]=='JJ' or arr[0][indeks+10][1]=='PR' or arr[0][indeks+10][1]=='NND' or arr[0][indeks+10][1]=='RB') 
        and (arr[0][indeks+11][1]=='CD' or arr[0][indeks+11][0]=='Seorang' or arr[0][indeks+11][0]=='Seseorang' or arr[0][indeks+11][0]=='Empat' or arr[0][indeks+11][0]=='Lima' or arr[0][indeks+11][0]=='Enam' or arr[0][indeks+11][0]=='Tujuh' or arr[0][indeks+11][0]=='Delapan' or arr[0][indeks+11][0]=='Sembilan' or arr[0][indeks+11][0]=='Sepuluh')):
        return arr[0][indeks+11][0]
    elif(indeks-11>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='VB' or arr[0][indeks-9][1]=='IN' or arr[0][indeks-9][1]=='NN' or arr[0][indeks-9][1]=='NNP' or arr[0][indeks-9][1]=='SC' or arr[0][indeks-9][1]=='JJ' or arr[0][indeks-9][1]=='PR'  or arr[0][indeks-9][1]=='NND'  or arr[0][indeks-9][1]=='RB') 
        and (arr[0][indeks-10][1]=='VB' or arr[0][indeks-10][1]=='IN' or arr[0][indeks-10][1]=='NN' or arr[0][indeks-10][1]=='NNP' or arr[0][indeks-10][1]=='SC' or arr[0][indeks-10][1]=='JJ' or arr[0][indeks-10][1]=='PR' or arr[0][indeks-10][1]=='NND' or arr[0][indeks-10][1]=='RB') 
        and (arr[0][indeks-11][1]=='CD' or arr[0][indeks-11][0]=='Seorang' or arr[0][indeks-11][0]=='Seseorang' or arr[0][indeks-11][0]=='Empat' or arr[0][indeks-11][0]=='Lima' or arr[0][indeks-11][0]=='Enam' or arr[0][indeks-11][0]=='Tujuh' or arr[0][indeks-11][0]=='Delapan' or arr[0][indeks-11][0]=='Sembilan' or arr[0][indeks-11][0]=='Sepuluh')):
        return arr[0][indeks-11][0]
    elif(indeks+12<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='VB' or arr[0][indeks+9][1]=='IN' or arr[0][indeks+9][1]=='NN' or arr[0][indeks+9][1]=='NNP' or arr[0][indeks+9][1]=='SC' or arr[0][indeks+9][1]=='JJ' or arr[0][indeks+9][1]=='PR'  or arr[0][indeks+9][1]=='NND'  or arr[0][indeks+9][1]=='RB') 
        and (arr[0][indeks+10][1]=='VB' or arr[0][indeks+10][1]=='IN' or arr[0][indeks+10][1]=='NN' or arr[0][indeks+10][1]=='NNP' or arr[0][indeks+10][1]=='SC' or arr[0][indeks+10][1]=='JJ' or arr[0][indeks+10][1]=='PR' or arr[0][indeks+10][1]=='NND' or arr[0][indeks+10][1]=='RB') 
        and (arr[0][indeks+11][1]=='VB' or arr[0][indeks+11][1]=='IN' or arr[0][indeks+11][1]=='NN' or arr[0][indeks+11][1]=='NNP' or arr[0][indeks+11][1]=='SC' or arr[0][indeks+11][1]=='JJ' or arr[0][indeks+11][1]=='PR' or arr[0][indeks+11][1]=='NND' or arr[0][indeks+11][1]=='RB') 
        and (arr[0][indeks+12][1]=='CD' or arr[0][indeks+12][0]=='Seorang' or arr[0][indeks+12][0]=='Seseorang' or arr[0][indeks+12][0]=='Empat' or arr[0][indeks+12][0]=='Lima' or arr[0][indeks+12][0]=='Enam' or arr[0][indeks+12][0]=='Tujuh' or arr[0][indeks+12][0]=='Delapan' or arr[0][indeks+12][0]=='Sembilan' or arr[0][indeks+12][0]=='Sepuluh')):
        return arr[0][indeks+12][0]
    elif(indeks-12>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='VB' or arr[0][indeks-9][1]=='IN' or arr[0][indeks-9][1]=='NN' or arr[0][indeks-9][1]=='NNP' or arr[0][indeks-9][1]=='SC' or arr[0][indeks-9][1]=='JJ' or arr[0][indeks-9][1]=='PR'  or arr[0][indeks-9][1]=='NND'  or arr[0][indeks-9][1]=='RB') 
        and (arr[0][indeks-10][1]=='VB' or arr[0][indeks-10][1]=='IN' or arr[0][indeks-10][1]=='NN' or arr[0][indeks-10][1]=='NNP' or arr[0][indeks-10][1]=='SC' or arr[0][indeks-10][1]=='JJ' or arr[0][indeks-10][1]=='PR' or arr[0][indeks-10][1]=='NND' or arr[0][indeks-10][1]=='RB') 
        and (arr[0][indeks-11][1]=='VB' or arr[0][indeks-11][1]=='IN' or arr[0][indeks-11][1]=='NN' or arr[0][indeks-11][1]=='NNP' or arr[0][indeks-11][1]=='SC' or arr[0][indeks-11][1]=='JJ' or arr[0][indeks-11][1]=='PR' or arr[0][indeks-11][1]=='NND' or arr[0][indeks-11][1]=='RB') 
        and (arr[0][indeks-12][1]=='CD' or arr[0][indeks-12][0]=='Seorang' or arr[0][indeks-12][0]=='Seseorang' or arr[0][indeks-12][0]=='Empat' or arr[0][indeks-12][0]=='Lima' or arr[0][indeks-12][0]=='Enam' or arr[0][indeks-12][0]=='Tujuh' or arr[0][indeks-12][0]=='Delapan' or arr[0][indeks-12][0]=='Sembilan' or arr[0][indeks-12][0]=='Sepuluh')):
        return arr[0][indeks-12][0]
    elif(indeks+13<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='VB' or arr[0][indeks+9][1]=='IN' or arr[0][indeks+9][1]=='NN' or arr[0][indeks+9][1]=='NNP' or arr[0][indeks+9][1]=='SC' or arr[0][indeks+9][1]=='JJ' or arr[0][indeks+9][1]=='PR'  or arr[0][indeks+9][1]=='NND'  or arr[0][indeks+9][1]=='RB') 
        and (arr[0][indeks+10][1]=='VB' or arr[0][indeks+10][1]=='IN' or arr[0][indeks+10][1]=='NN' or arr[0][indeks+10][1]=='NNP' or arr[0][indeks+10][1]=='SC' or arr[0][indeks+10][1]=='JJ' or arr[0][indeks+10][1]=='PR' or arr[0][indeks+10][1]=='NND' or arr[0][indeks+10][1]=='RB') 
        and (arr[0][indeks+11][1]=='VB' or arr[0][indeks+11][1]=='IN' or arr[0][indeks+11][1]=='NN' or arr[0][indeks+11][1]=='NNP' or arr[0][indeks+11][1]=='SC' or arr[0][indeks+11][1]=='JJ' or arr[0][indeks+11][1]=='PR' or arr[0][indeks+11][1]=='NND' or arr[0][indeks+11][1]=='RB') 
        and (arr[0][indeks+12][1]=='VB' or arr[0][indeks+12][1]=='IN' or arr[0][indeks+12][1]=='NN' or arr[0][indeks+12][1]=='NNP' or arr[0][indeks+12][1]=='SC' or arr[0][indeks+12][1]=='JJ' or arr[0][indeks+12][1]=='PR' or arr[0][indeks+12][1]=='NND' or arr[0][indeks+12][1]=='RB') 
        and (arr[0][indeks+13][1]=='CD' or arr[0][indeks+13][0]=='Seorang' or arr[0][indeks+13][0]=='Seseorang' or arr[0][indeks+13][0]=='Empat' or arr[0][indeks+13][0]=='Lima' or arr[0][indeks+13][0]=='Enam' or arr[0][indeks+13][0]=='Tujuh' or arr[0][indeks+13][0]=='Delapan' or arr[0][indeks+13][0]=='Sembilan' or arr[0][indeks+13][0]=='Sepuluh')):
        return arr[0][indeks+13][0]
    elif(indeks-13>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='VB' or arr[0][indeks-9][1]=='IN' or arr[0][indeks-9][1]=='NN' or arr[0][indeks-9][1]=='NNP' or arr[0][indeks-9][1]=='SC' or arr[0][indeks-9][1]=='JJ' or arr[0][indeks-9][1]=='PR'  or arr[0][indeks-9][1]=='NND'  or arr[0][indeks-9][1]=='RB') 
        and (arr[0][indeks-10][1]=='VB' or arr[0][indeks-10][1]=='IN' or arr[0][indeks-10][1]=='NN' or arr[0][indeks-10][1]=='NNP' or arr[0][indeks-10][1]=='SC' or arr[0][indeks-10][1]=='JJ' or arr[0][indeks-10][1]=='PR' or arr[0][indeks-10][1]=='NND' or arr[0][indeks-10][1]=='RB') 
        and (arr[0][indeks-11][1]=='VB' or arr[0][indeks-11][1]=='IN' or arr[0][indeks-11][1]=='NN' or arr[0][indeks-11][1]=='NNP' or arr[0][indeks-11][1]=='SC' or arr[0][indeks-11][1]=='JJ' or arr[0][indeks-11][1]=='PR' or arr[0][indeks-11][1]=='NND' or arr[0][indeks-11][1]=='RB') 
        and (arr[0][indeks-12][1]=='VB' or arr[0][indeks-12][1]=='IN' or arr[0][indeks-12][1]=='NN' or arr[0][indeks-12][1]=='NNP' or arr[0][indeks-12][1]=='SC' or arr[0][indeks-12][1]=='JJ' or arr[0][indeks-12][1]=='PR' or arr[0][indeks-12][1]=='NND' or arr[0][indeks-12][1]=='RB') 
        and (arr[0][indeks-13][1]=='CD' or arr[0][indeks-13][0]=='Seorang' or arr[0][indeks-13][0]=='Seseorang' or arr[0][indeks-13][0]=='Empat' or arr[0][indeks-13][0]=='Lima' or arr[0][indeks-13][0]=='Enam' or arr[0][indeks-13][0]=='Tujuh' or arr[0][indeks-13][0]=='Delapan' or arr[0][indeks-13][0]=='Sembilan' or arr[0][indeks-13][0]=='Sepuluh')):
        return arr[0][indeks-13][0]
    elif(indeks+14<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='VB' or arr[0][indeks+9][1]=='IN' or arr[0][indeks+9][1]=='NN' or arr[0][indeks+9][1]=='NNP' or arr[0][indeks+9][1]=='SC' or arr[0][indeks+9][1]=='JJ' or arr[0][indeks+9][1]=='PR'  or arr[0][indeks+9][1]=='NND'  or arr[0][indeks+9][1]=='RB') 
        and (arr[0][indeks+10][1]=='VB' or arr[0][indeks+10][1]=='IN' or arr[0][indeks+10][1]=='NN' or arr[0][indeks+10][1]=='NNP' or arr[0][indeks+10][1]=='SC' or arr[0][indeks+10][1]=='JJ' or arr[0][indeks+10][1]=='PR' or arr[0][indeks+10][1]=='NND' or arr[0][indeks+10][1]=='RB') 
        and (arr[0][indeks+11][1]=='VB' or arr[0][indeks+11][1]=='IN' or arr[0][indeks+11][1]=='NN' or arr[0][indeks+11][1]=='NNP' or arr[0][indeks+11][1]=='SC' or arr[0][indeks+11][1]=='JJ' or arr[0][indeks+11][1]=='PR' or arr[0][indeks+11][1]=='NND' or arr[0][indeks+11][1]=='RB') 
        and (arr[0][indeks+12][1]=='VB' or arr[0][indeks+12][1]=='IN' or arr[0][indeks+12][1]=='NN' or arr[0][indeks+12][1]=='NNP' or arr[0][indeks+12][1]=='SC' or arr[0][indeks+12][1]=='JJ' or arr[0][indeks+12][1]=='PR' or arr[0][indeks+12][1]=='NND' or arr[0][indeks+12][1]=='RB') 
        and (arr[0][indeks+13][1]=='VB' or arr[0][indeks+13][1]=='IN' or arr[0][indeks+13][1]=='NN' or arr[0][indeks+13][1]=='NNP' or arr[0][indeks+13][1]=='SC' or arr[0][indeks+13][1]=='JJ' or arr[0][indeks+13][1]=='PR' or arr[0][indeks+13][1]=='NND' or arr[0][indeks+13][1]=='RB') 
        and (arr[0][indeks+14][1]=='CD' or arr[0][indeks+14][0]=='Seorang' or arr[0][indeks+14][0]=='Seseorang' or arr[0][indeks+14][0]=='Empat' or arr[0][indeks+14][0]=='Lima' or arr[0][indeks+14][0]=='Enam' or arr[0][indeks+14][0]=='Tujuh' or arr[0][indeks+14][0]=='Delapan' or arr[0][indeks+14][0]=='Sembilan' or arr[0][indeks+14][0]=='Sepuluh')):
        return arr[0][indeks+14][0]
    elif(indeks-14>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='VB' or arr[0][indeks-9][1]=='IN' or arr[0][indeks-9][1]=='NN' or arr[0][indeks-9][1]=='NNP' or arr[0][indeks-9][1]=='SC' or arr[0][indeks-9][1]=='JJ' or arr[0][indeks-9][1]=='PR'  or arr[0][indeks-9][1]=='NND'  or arr[0][indeks-9][1]=='RB') 
        and (arr[0][indeks-10][1]=='VB' or arr[0][indeks-10][1]=='IN' or arr[0][indeks-10][1]=='NN' or arr[0][indeks-10][1]=='NNP' or arr[0][indeks-10][1]=='SC' or arr[0][indeks-10][1]=='JJ' or arr[0][indeks-10][1]=='PR' or arr[0][indeks-10][1]=='NND' or arr[0][indeks-10][1]=='RB') 
        and (arr[0][indeks-11][1]=='VB' or arr[0][indeks-11][1]=='IN' or arr[0][indeks-11][1]=='NN' or arr[0][indeks-11][1]=='NNP' or arr[0][indeks-11][1]=='SC' or arr[0][indeks-11][1]=='JJ' or arr[0][indeks-11][1]=='PR' or arr[0][indeks-11][1]=='NND' or arr[0][indeks-11][1]=='RB') 
        and (arr[0][indeks-12][1]=='VB' or arr[0][indeks-12][1]=='IN' or arr[0][indeks-12][1]=='NN' or arr[0][indeks-12][1]=='NNP' or arr[0][indeks-12][1]=='SC' or arr[0][indeks-12][1]=='JJ' or arr[0][indeks-12][1]=='PR' or arr[0][indeks-12][1]=='NND' or arr[0][indeks-12][1]=='RB') 
        and (arr[0][indeks-13][1]=='VB' or arr[0][indeks-13][1]=='IN' or arr[0][indeks-13][1]=='NN' or arr[0][indeks-13][1]=='NNP' or arr[0][indeks-13][1]=='SC' or arr[0][indeks-13][1]=='JJ' or arr[0][indeks-13][1]=='PR' or arr[0][indeks-13][1]=='NND' or arr[0][indeks-13][1]=='RB') 
        and (arr[0][indeks-14][1]=='CD' or arr[0][indeks-14][0]=='Seorang' or arr[0][indeks-14][0]=='Seseorang' or arr[0][indeks-14][0]=='Empat' or arr[0][indeks-14][0]=='Lima' or arr[0][indeks-14][0]=='Enam' or arr[0][indeks-14][0]=='Tujuh' or arr[0][indeks-14][0]=='Delapan' or arr[0][indeks-14][0]=='Sembilan' or arr[0][indeks-14][0]=='Sepuluh')):
        return arr[0][indeks-14][0]
    elif(indeks+15<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='VB' or arr[0][indeks+9][1]=='IN' or arr[0][indeks+9][1]=='NN' or arr[0][indeks+9][1]=='NNP' or arr[0][indeks+9][1]=='SC' or arr[0][indeks+9][1]=='JJ' or arr[0][indeks+9][1]=='PR'  or arr[0][indeks+9][1]=='NND'  or arr[0][indeks+9][1]=='RB') 
        and (arr[0][indeks+10][1]=='VB' or arr[0][indeks+10][1]=='IN' or arr[0][indeks+10][1]=='NN' or arr[0][indeks+10][1]=='NNP' or arr[0][indeks+10][1]=='SC' or arr[0][indeks+10][1]=='JJ' or arr[0][indeks+10][1]=='PR' or arr[0][indeks+10][1]=='NND' or arr[0][indeks+10][1]=='RB') 
        and (arr[0][indeks+11][1]=='VB' or arr[0][indeks+11][1]=='IN' or arr[0][indeks+11][1]=='NN' or arr[0][indeks+11][1]=='NNP' or arr[0][indeks+11][1]=='SC' or arr[0][indeks+11][1]=='JJ' or arr[0][indeks+11][1]=='PR' or arr[0][indeks+11][1]=='NND' or arr[0][indeks+11][1]=='RB') 
        and (arr[0][indeks+12][1]=='VB' or arr[0][indeks+12][1]=='IN' or arr[0][indeks+12][1]=='NN' or arr[0][indeks+12][1]=='NNP' or arr[0][indeks+12][1]=='SC' or arr[0][indeks+12][1]=='JJ' or arr[0][indeks+12][1]=='PR' or arr[0][indeks+12][1]=='NND' or arr[0][indeks+12][1]=='RB') 
        and (arr[0][indeks+13][1]=='VB' or arr[0][indeks+13][1]=='IN' or arr[0][indeks+13][1]=='NN' or arr[0][indeks+13][1]=='NNP' or arr[0][indeks+13][1]=='SC' or arr[0][indeks+13][1]=='JJ' or arr[0][indeks+13][1]=='PR' or arr[0][indeks+13][1]=='NND' or arr[0][indeks+13][1]=='RB') 
        and (arr[0][indeks+14][1]=='VB' or arr[0][indeks+14][1]=='IN' or arr[0][indeks+14][1]=='NN' or arr[0][indeks+14][1]=='NNP' or arr[0][indeks+14][1]=='SC' or arr[0][indeks+14][1]=='JJ' or arr[0][indeks+14][1]=='PR' or arr[0][indeks+14][1]=='NND' or arr[0][indeks+14][1]=='RB') 
        and (arr[0][indeks+15][1]=='CD' or arr[0][indeks+15][0]=='Seorang' or arr[0][indeks+15][0]=='Seseorang' or arr[0][indeks+15][0]=='Empat' or arr[0][indeks+15][0]=='Lima' or arr[0][indeks+15][0]=='Enam' or arr[0][indeks+15][0]=='Tujuh' or arr[0][indeks+15][0]=='Delapan' or arr[0][indeks+15][0]=='Sembilan' or arr[0][indeks+15][0]=='Sepuluh')):
        return arr[0][indeks+15][0]
    elif(indeks-15>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='VB' or arr[0][indeks-9][1]=='IN' or arr[0][indeks-9][1]=='NN' or arr[0][indeks-9][1]=='NNP' or arr[0][indeks-9][1]=='SC' or arr[0][indeks-9][1]=='JJ' or arr[0][indeks-9][1]=='PR'  or arr[0][indeks-9][1]=='NND'  or arr[0][indeks-9][1]=='RB') 
        and (arr[0][indeks-10][1]=='VB' or arr[0][indeks-10][1]=='IN' or arr[0][indeks-10][1]=='NN' or arr[0][indeks-10][1]=='NNP' or arr[0][indeks-10][1]=='SC' or arr[0][indeks-10][1]=='JJ' or arr[0][indeks-10][1]=='PR' or arr[0][indeks-10][1]=='NND' or arr[0][indeks-10][1]=='RB') 
        and (arr[0][indeks-11][1]=='VB' or arr[0][indeks-11][1]=='IN' or arr[0][indeks-11][1]=='NN' or arr[0][indeks-11][1]=='NNP' or arr[0][indeks-11][1]=='SC' or arr[0][indeks-11][1]=='JJ' or arr[0][indeks-11][1]=='PR' or arr[0][indeks-11][1]=='NND' or arr[0][indeks-11][1]=='RB') 
        and (arr[0][indeks-12][1]=='VB' or arr[0][indeks-12][1]=='IN' or arr[0][indeks-12][1]=='NN' or arr[0][indeks-12][1]=='NNP' or arr[0][indeks-12][1]=='SC' or arr[0][indeks-12][1]=='JJ' or arr[0][indeks-12][1]=='PR' or arr[0][indeks-12][1]=='NND' or arr[0][indeks-12][1]=='RB') 
        and (arr[0][indeks-13][1]=='VB' or arr[0][indeks-13][1]=='IN' or arr[0][indeks-13][1]=='NN' or arr[0][indeks-13][1]=='NNP' or arr[0][indeks-13][1]=='SC' or arr[0][indeks-13][1]=='JJ' or arr[0][indeks-13][1]=='PR' or arr[0][indeks-13][1]=='NND' or arr[0][indeks-13][1]=='RB') 
        and (arr[0][indeks-14][1]=='VB' or arr[0][indeks-14][1]=='IN' or arr[0][indeks-14][1]=='NN' or arr[0][indeks-14][1]=='NNP' or arr[0][indeks-14][1]=='SC' or arr[0][indeks-14][1]=='JJ' or arr[0][indeks-14][1]=='PR' or arr[0][indeks-14][1]=='NND' or arr[0][indeks-14][1]=='RB') 
        and (arr[0][indeks-15][1]=='CD' or arr[0][indeks-15][0]=='Seorang' or arr[0][indeks-15][0]=='Seseorang' or arr[0][indeks-15][0]=='Empat' or arr[0][indeks-15][0]=='Lima' or arr[0][indeks-15][0]=='Enam' or arr[0][indeks-15][0]=='Tujuh' or arr[0][indeks-15][0]=='Delapan' or arr[0][indeks-15][0]=='Sembilan' or arr[0][indeks-15][0]=='Sepuluh')):
        return arr[0][indeks-15][0]
    else:
        return 1


In [11]:
def indeksKe(arr,indikator):
    for x in range(0, len(arr[0])):
        for y in indikator:
            if(arr[0][x][0]==y):
    #             return x
    # return 0
                # print(arr[0][x][0],arr[0][x+1][0])
                depan =cek_CD_depan(arr, x)
                belakang =cek_CD_belakang(arr, x)
                if(belakang==0 and depan!=0):
                    sentence =getSentence(x,depan[0],arr)
                    return (x,depan[1],sentence)
                elif(depan==0 and belakang!=0):
                    sentence =getSentence(x,belakang[0],arr)
                    return (x,belakang[1],sentence)
                else:
                    titik = getTitik(x,arr)
                    sentence = getSentenceNotCD(titik[0],titik[1],x,arr)
                    return(x,1,sentence)

    return(0,0,'-')

In [13]:
def teslist(indeks, arr):
    front = 0
    back = 0
    if(indeks< len(arr[0])-1):
        x = indeks+1
    else:
        x = indeks
    if(indeks> 0):
        y = indeks-1
    else:
        y = indeks

    while True:
        # print (x,y)
        front = arr[0][y][1]
        back = arr[0][x][1]
        if(front=='CD'):
            return arr[0][y][0]
        elif(back=='CD'):
            return arr[0][x][0]
        
        
        if((x== len(arr[0])-1 or arr[0][x][1]=='Z') and (y==0 or arr[0][y][1]=='Z')):
            return 1
        elif((x== len(arr[0])-1 or arr[0][x][1]=='Z') and (y!=0 and arr[0][y][1]!='Z')):
            x=x
            y=y-1
        elif((x!= len(arr[0])-1 and arr[0][x][1]!='Z') and (y==0 or arr[0][y][1]=='Z')):
            x=x+1
            y=y
        else:
            x=x+1
            y=y-1

In [53]:
hasilAkhir=[]
struktur=[]
for x in range(0,len(df)):
    arr =[] 
    berita =preprocessing(df.iloc[x,1])
    arr.append(berita)
    hasil = ct.tag_sents(arr)

    #ges yang di bawah ini blom fix karena aku masih masukin dia ngambil structurenya sama ada ngetes pake cara satu lagi 

    # death_indeks,death_value,death_sentence= indeksKe(hasil,['meninggal','mati','tewas','meregang','menewaskan'])
    death_indeks,death_value,death_sentence= indeksKe(hasil,['meninggal','mati','tewas','nyawa','menewaskan']) #kematian
    injury_indeks,injury_value,injury_sentence= indeksKe(hasil,['luka-luka','luka','melukai'])

    if(death_indeks==0 and death_value==0):
        mati = 0
        tesMati=0
    else:
        mati = struktur1(hasil,death_indeks)
        tesMati = teslist(death_indeks, hasil)

    if(injury_indeks==0 and injury_value==0):
        luka = 0
        tesLuka = 0
    else:
        luka = struktur1(hasil,injury_indeks)
        tesLuka = teslist(injury_indeks, hasil)
    
    # print(mati,luka)

    data = [x,df.iloc[x,1],death_indeks,mati,tesMati,death_sentence,injury_indeks, luka,tesLuka,injury_sentence]
    hasilAkhir.append(data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84


In [51]:
writer = pd.DataFrame(hasilAkhir, columns=['No Document','Description', 'indeks Mati','mati','Tes Mati', 'struktur','Indeks Luka', 'luka','Tes Lukka','struktur'])
writer.to_csv('hasilStrucktureList25_5.csv', index=False, sep=',')